In [ ]:
from classifier import ManifoldClassifier
import numpy as np
import pandas as pd
import utils
import xarray as xr
import matplotlib.pyplot as plt
from core import Core, use_dim_type, analyze_dist_type, score_agg_type
from component import ComponentGroups
from sklearn.model_selection import train_test_split

In [ ]:
data_AD = np.load("data/AD.npy")
data_NL = np.load("data/Normal.npy")

protein_coding_ID = pd.read_csv('data/protein_coding_ID.csv', index_col=0)
protein_coding_indices = protein_coding_ID.index.to_numpy()

X_np = np.concatenate([data_AD, data_NL], axis=0)
y_np = np.concatenate([np.ones(len(data_AD), dtype=np.int8),
                    np.zeros(len(data_NL), dtype=np.int8)], axis=0)

X = xr.DataArray(X_np, dims=['person', 'gene', 'section'], coords={'person': np.arange(X_np.shape[0])})
y = xr.DataArray(y_np, dims=['person'])

print(X.shape, y.shape)

In [ ]:
ManifoldClassifier.manifolds_cache = Core(data=X.values, dataType="all").fit_manifolds()

In [ ]:
n_clusters, n_genes = (4, 1), 5
score_agg_method: score_agg_type = "min"
dist_type: analyze_dist_type = "default"
use_dim: use_dim_type = "4D4D"


def feature_getter(cgs: ComponentGroups) -> np.ndarray:
    return np.hstack([cgs.total_curvatures[:, None], cgs.areas[:, None]])

def typical_analyzer(cgs: ComponentGroups, people: np.ndarray, attr_mean: np.ndarray):
    total_curvature, mean_area = attr_mean
    typical_person_index = np.argmin(
        (cgs.total_curvatures[people] - total_curvature) ** 2
        + (cgs.areas[people] - mean_area) ** 2
    )
    return people[typical_person_index]


classifier = ManifoldClassifier(
    use_dim=use_dim,
    fit_manifold_config=dict(),
    cluster_configs=[
        dict(n_group=n_clusters[0], feature_getter=feature_getter),
        dict(n_group=n_clusters[1], feature_getter=feature_getter)
    ],
    analyze_config=dict(top_k=n_genes,
                        typical_analyzer=typical_analyzer,
                        selectable_indices=protein_coding_indices),
    classify_config=dict(score_agg_method=score_agg_method,
                         dist_type=dist_type)
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=0)
classifier.fit(X_train, y_train)

In [ ]:
utils.show_classification_result(y_train, classifier.predict_proba(X_train), names=['Normal', 'AD'])
utils.show_classification_result(y_test, classifier.predict_proba(X_test), names=['Normal', 'AD'])

In [ ]:
labels = ['Normal', 'AD']

n1 = classifier.cluster_configs[0]['n_group']
n2 = classifier.cluster_configs[1]['n_group']

title = f'2 Class Classification Train Results for {labels[0]}/{labels[1]}'
print(title)
utils.show_classification_result(y_train, classifier.predict_proba(X_train), normalize='pred', names=labels)
plt.suptitle(title)
plt.show()

title = f'2 Class Classification Test Results for {labels[0]}/{labels[1]}'
print(title)
utils.show_classification_result(y_test, classifier.predict_proba(X_test), normalize='pred', names=labels)
plt.suptitle(title)
plt.show()

title = f'{n1+n2} Class Classification Results for AD/Normal'
pred_scores = classifier.mixed_core.classify_with_typical(use_dim=use_dim, data=X_train.values, **classifier.classify_config)
assert pred_scores.shape == (len(X_train), n1+n2)
true_label = np.empty(len(X_train), dtype=np.int8)
true_label[y_train==0] = np.array(classifier.cores[0].group_result.person2group)
true_label[y_train==1] = np.array(classifier.cores[1].group_result.person2group) + n1
print(title)
utils.show_classification_result(true_label, pred_scores, normalize=None)
plt.suptitle(title)

plt.figure(figsize=(6, 2))
class_counts = np.bincount(true_label)
plt.bar(range(n1), class_counts[:n1], label=labels[0])
plt.bar(range(n1, n1 + n2), class_counts[n1:], color='#896989',  label=labels[1])
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Frequency of Each Class in true_label')
plt.legend()
plt.show()

if n1 > 1:
    title = f'{n1} Class Classification Results for {labels[0]}s'
    pred_scores = classifier.cores[0].classify_with_typical(use_dim=use_dim, data=X_train[y_train==0].values, **classifier.classify_config)
    true_label = np.array(classifier.cores[0].group_result.person2group)
    utils.show_classification_result(true_label, pred_scores, normalize='pred')
    plt.suptitle(title)

    plt.figure(figsize=(3, 2))
    utils.plot_bincount(true_label, title=f'Frequency of Each {labels[0]} Class', normalize=True)
    plt.show()


if n2 > 1:
    title = f'{n2} Class Classification Results for {labels[1]}s'
    pred_scores = classifier.cores[1].classify_with_typical(use_dim=use_dim, data=X_train[y_train==1].values, **classifier.classify_config)
    true_label = np.array(classifier.cores[1].group_result.person2group)
    utils.show_classification_result(true_label, pred_scores, normalize='pred')
    plt.suptitle(title)

    plt.figure(figsize=(3, 2))
    utils.plot_bincount(true_label, title=f'Frequency of Each {labels[1]} Class', normalize=True)
    plt.show()


In [ ]:
print("Selected genes:")
classifier.fit(X, y)
print(protein_coding_ID.loc[classifier.typical_indices])